In [1]:
# installing required packages for the current runtime

!pip install wget
!pip install jsonlines

In [0]:
# importing required packages

import tweepy
import jsonlines
import json
import csv
import wget
import pandas as pd

In [0]:
# user keys and tokens

consumer_key = ''
consumer_secret_key = ''
access_token = ''
access_token_secret = ''

In [0]:
def process_media(entities):
  """accepts a tweet entities object,
  returns the number of images in the tweet"""
  
  ret = 0
  
  if entities.get('media'):
    ret += len(entities.get('media'))
  
  return ret if ret else None

In [0]:
def get_all_tweets(user_handle):
  """accepts a user screen_name,
  returns a list of tweepy status objects"""
  
  # user authorization
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth)
  
  # loading tweets in batches of size of 200
  all_tweets = []
  new_tweets = api.user_timeline(screen_name = user_handle, count = 200)
  all_tweets.extend(new_tweets)
  oldest_tweet = all_tweets[-1].id - 1
  while len(new_tweets) > 0:
    new_tweets = api.user_timeline(screen_name = user_handle, count = 200, max_id = oldest_tweet)
    all_tweets.extend(new_tweets)
    oldest_tweet = all_tweets[-1].id - 1
    
  return all_tweets

In [6]:
if __name__ == '__main__':
  
  # fetching all the tweets of a given user
  all_tweets = get_all_tweets(input("Enter handle : "))
  
  # writing all the tweets to jsonlines file
  writer = jsonlines.open('tweets.jsonl', mode = 'w')
  for tweet in all_tweets:
    writer.write(tweet._json)

Enter handle : midasIIITD


In [0]:
# extracting required information to a list of lists

# required columns
columns = ['id', 'created_at', 'text', 'retweet_count', 'favorite_count', 'entities']

data = []

with jsonlines.open('tweets.jsonl') as reader:
   for obj in reader:
     selected_row = []
     for item in columns:
       selected_row.append(obj[item])
     data.append(selected_row)

In [8]:
# converting the list into a Dataframe with required columns

table = pd.DataFrame(data, columns = columns)
table.head(10)

,id,created_at,text,retweet_count,favorite_count,entities
0,1109888617302753280,Sun Mar 24 18:44:01 +0000 2019,The last date for submitting a solution for th...,3,7,"{'hashtags': [], 'symbols': [], 'user_mentions..."
1,1109884092101754880,Sun Mar 24 18:26:02 +0000 2019,RT @IIITDelhi: @IIITDelhi invites application ...,4,0,"{'hashtags': [], 'symbols': [], 'user_mentions..."
2,1109780512330022918,Sun Mar 24 11:34:27 +0000 2019,One more week is left to submit the workshop p...,0,4,"{'hashtags': [], 'symbols': [], 'user_mentions..."
3,1109702288136781826,Sun Mar 24 06:23:37 +0000 2019,RT @IEEEBigMM19: We are honored to have Dr. Ch...,5,0,"{'hashtags': [], 'symbols': [], 'user_mentions..."
4,1109702192439582722,Sun Mar 24 06:23:14 +0000 2019,RT @IEEEBigMM19: Distinguished researchers Dr....,3,0,"{'hashtags': [], 'symbols': [], 'user_mentions..."
5,1108281874164658182,Wed Mar 20 08:19:24 +0000 2019,@IEEEBigMM19 is also available on Facebook now...,1,1,"{'hashtags': [], 'symbols': [], 'user_mentions..."
6,1108196492139999233,Wed Mar 20 02:40:07 +0000 2019,RT @IEEEBigMM19: BigMM 2019 : IEEE BigMM 2019 ...,5,0,"{'hashtags': [], 'symbols': [], 'user_mentions..."
7,1107468609914208256,Mon Mar 18 02:27:47 +0000 2019,BigMM 2019 : IEEE BigMM 2019 – Call for Worksh...,3,6,"{'hashtags': [], 'symbols': [], 'user_mentions..."
8,1107285980082569218,Sun Mar 17 14:22:04 +0000 2019,"Congratulations @midasIIITD team, Rohan, Prady...",4,15,"{'hashtags': [], 'symbols': [], 'user_mentions..."
9,1106919781616091136,Sat Mar 16 14:06:56 +0000 2019,We have emailed the task details to all shortl...,0,7,"{'hashtags': [], 'symbols': [], 'user_mentions..."


In [9]:
# extracting images from entities objects

table['image_count'] = table['entities'].apply(lambda x : process_media(x))
table.head(10)

,id,created_at,text,retweet_count,favorite_count,entities,image_count
0,1109888617302753280,Sun Mar 24 18:44:01 +0000 2019,The last date for submitting a solution for th...,3,7,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
1,1109884092101754880,Sun Mar 24 18:26:02 +0000 2019,RT @IIITDelhi: @IIITDelhi invites application ...,4,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
2,1109780512330022918,Sun Mar 24 11:34:27 +0000 2019,One more week is left to submit the workshop p...,0,4,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
3,1109702288136781826,Sun Mar 24 06:23:37 +0000 2019,RT @IEEEBigMM19: We are honored to have Dr. Ch...,5,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
4,1109702192439582722,Sun Mar 24 06:23:14 +0000 2019,RT @IEEEBigMM19: Distinguished researchers Dr....,3,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
5,1108281874164658182,Wed Mar 20 08:19:24 +0000 2019,@IEEEBigMM19 is also available on Facebook now...,1,1,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
6,1108196492139999233,Wed Mar 20 02:40:07 +0000 2019,RT @IEEEBigMM19: BigMM 2019 : IEEE BigMM 2019 ...,5,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
7,1107468609914208256,Mon Mar 18 02:27:47 +0000 2019,BigMM 2019 : IEEE BigMM 2019 – Call for Worksh...,3,6,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
8,1107285980082569218,Sun Mar 17 14:22:04 +0000 2019,"Congratulations @midasIIITD team, Rohan, Prady...",4,15,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
9,1106919781616091136,Sat Mar 16 14:06:56 +0000 2019,We have emailed the task details to all shortl...,0,7,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN


In [10]:
# removing the useless columns and printing the table

table.drop(['entities'], axis = 1, inplace = True)
table.head(table.shape[0])

,id,created_at,text,retweet_count,favorite_count,image_count
0,1109888617302753280,Sun Mar 24 18:44:01 +0000 2019,The last date for submitting a solution for th...,3,7,NaN
1,1109884092101754880,Sun Mar 24 18:26:02 +0000 2019,RT @IIITDelhi: @IIITDelhi invites application ...,4,0,NaN
2,1109780512330022918,Sun Mar 24 11:34:27 +0000 2019,One more week is left to submit the workshop p...,0,4,NaN
3,1109702288136781826,Sun Mar 24 06:23:37 +0000 2019,RT @IEEEBigMM19: We are honored to have Dr. Ch...,5,0,NaN
4,1109702192439582722,Sun Mar 24 06:23:14 +0000 2019,RT @IEEEBigMM19: Distinguished researchers Dr....,3,0,NaN
5,1108281874164658182,Wed Mar 20 08:19:24 +0000 2019,@IEEEBigMM19 is also available on Facebook now...,1,1,NaN
6,1108196492139999233,Wed Mar 20 02:40:07 +0000 2019,RT @IEEEBigMM19: BigMM 2019 : IEEE BigMM 2019 ...,5,0,NaN
7,1107468609914208256,Mon Mar 18 02:27:47 +0000 2019,BigMM 2019 : IEEE BigMM 2019 – Call for Worksh...,3,6,NaN
8,1107285980082569218,Sun Mar 17 14:22:04 +0000 2019,"Congratulations @midasIIITD team, Rohan, Prady...",4,15,NaN
9,1106919781616091136,Sat Mar 16 14:06:56 +0000 2019,We have emailed the task details to all shortl...,0,7,NaN
